<a href="https://colab.research.google.com/github/123EFD/EY-AI-Data/blob/main/Benchmark_EY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Water Quality Prediction: Benchmark Notebook

## Challenge Overview

<p align="justify">
Welcome to the EY AI & Data Challenge 2026!  
The objective of this challenge is to build a robust <strong> machine learning model </strong>capable of predicting water quality across various river locations in South Africa. In addition to accurate predictions, the model should also identify and emphasize the key factors that significantly influence water quality.
</p>

<p align="justify">
Participants will be provided with a dataset containing three water quality parameters <strong>Total Alkalinity</strong>, <strong>Electrical Conductance</strong>, and <strong>Dissolved Reactive Phosphorus</strong> collected between 2011 and 2015 from approximately 200 river locations across South Africa. Each data point includes the geographic coordinates (latitude and longitude) of the sampling site, the date of collection, and the corresponding water quality measurements.
</p>

<p align="justify">
Using this dataset, participants are expected to build a machine learning model to predict water quality parameters for a separate validation dataset, which includes locations from different regions not present in the training data. The challenge also encourages participants to explore feature importance and provide insights into the factors most strongly associated with variations in water quality.
</p>

<p align="justify">
This challenge is designed for participants with varying levels of experience in data science, remote sensing, and environmental analytics. It offers a valuable opportunity to apply machine learning techniques to real-world environmental data and contribute to advancing water quality monitoring using artificial intelligence.
</p>


<b>About the Notebook: </b><p align="justify"> <p>

<p align="justify"> In this notebook, we demonstrate a basic workflow that serves as a foundation for the challenge. The model has been developed to predict <b>water quality parameters</b> using features derived from the <b>Landsat</b> and <b>TerraClimate</b> datasets. Specifically, four spectral bands—<b>SWIR22</b> (Shortwave Infrared 2), <b>NIR</b> (Near Infrared), <b>Green</b>, and <b>SWIR16</b> (Shortwave Infrared 1)—were utilized from Landsat, along with derived spectral indices such as <b>NDMI</b> (Normalized Difference Moisture Index) and <b>MNDWI</b> (Modified Normalized Difference Water Index). In addition, the <b>PET</b> (Potential Evapotranspiration) variable was incorporated from the <b>TerraClimate</b> dataset to account for climatic influences on water quality. </p>

<p align="justify"> The dataset spans a five-year period from <b>2011 to 2015</b>. Using <b>API-based data extraction</b> methods, both Landsat and TerraClimate features were retrieved directly from the <a href="https://planetarycomputer.microsoft.com/">Microsoft Planetary Computer</a> portal. These combined spectral, index-based, and climatic features were used as predictors in a regression model to estimate three key water quality parameters: <b>Total Alkalinity (TA)</b>, <b>Electrical Conductance (EC)</b>, and <b>Dissolved Reactive Phosphorus (DRP)</b>.

</p> <p align="justify"> Please note that this notebook serves only as a starting point. Several assumptions were made during the data extraction and model development process, which you may find opportunities to improve upon. Participants are encouraged to explore additional features, enhance preprocessing techniques, or experiment with different regression algorithms to optimize predictive performance. </p>

## Load In Dependencies

To run this demonstration notebook, you will need to have the following packages imported below installed. This may take some time.  

In [ ]:
import os
# Move to the home directory or a safe existing path
os.chdir('/')
print(f"Current working directory: {os.getcwd()}")

Current working directory: /


In [ ]:
# Install geospatial, ML, and STAC dependencies
!pip install rioxarray xarray rasterio dask netCDF4 \
             scikit-learn pandas numpy matplotlib seaborn \
             pystac-client planetary-computer odc-stac tqdm scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.0/159.0 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.3 MB/s eta 0:00:00
  Attempting uninstall: xarray
    Found existing installation: xarray 2025.12.0
    Uninstalling xarray-2025.12.0:
      Successfully uninstalled xarray-2025.12.0


In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 1. Core Data Science & Viz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
from datetime import date

# 2. Geospatial & Raster handling
import xarray as xr
import rioxarray as rxr  # Extends xarray with geospatial capabilities
import rasterio
from rasterio.windows import Window
from scipy.spatial import cKDTree

# 3. Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

# 4. Planetary Computer & STAC API
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo

print("All modules imported successfully!")

All modules imported successfully!


## Response Variable

<p align="justify">
Before building the model, we first load the <b>water quality training dataset</b>. The curated dataset contains samples collected from various monitoring stations across the study region. Each record includes the geographical coordinates (Latitude and Longitude), the sample collection date, and the corresponding <b>measured values</b> for the three key water quality parameters — <b>Total Alkalinity (TA)</b>, <b>Electrical Conductance (EC)</b>, and <b>Dissolved Reactive Phosphorus (DRP)</b>.
</p>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
Water_Quality_df=pd.read_csv('/content/drive/MyDrive/terraclimate_features_validation.csv')
Water_Quality_df.head()

,Latitude,Longitude,Sample Date,pet
0,-32.043333,27.822778,01-09-2014,161.90001
1,-33.329167,26.077500,16-09-2015,177.60000
2,-32.991639,27.640028,07-05-2015,158.40001
3,-34.096389,24.439167,07-02-2012,130.00000
4,-32.000556,28.581667,01-10-2014,152.50000


## Predictor Variables

<p align="justify">
Now that we have our water quality dataset, the next step is to gather the predictor variables from the <b>Landsat</b> and <b>TerraClimate</b> datasets. In this notebook, we demonstrate how to <b>load previously extracted satellite and climate data</b> from separate files, rather than performing the extraction directly, which allows for a smoother and faster experience. Participants can refer to the dedicated extraction notebooks—one for Landsat and another for TerraClimate—to understand how the data was retrieved and processed, and they can also generate their own output CSV files if needed. Using these pre-extracted CSV files, this notebook focuses on loading the predictor features and running the subsequent analysis and model training efficiently.
</p>
<p align="justify">
For more detailed guidance on the original data extraction process, you can review the <a href="https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2#Example-Notebook">Landsat example notebook</a> and the <a href="https://planetarycomputer.microsoft.com/dataset/terraclimate#Example-Notebook">TerraClimate example notebook</a> available on the Planetary Computer portal.
</p>

<p align="justify">We have used selected spectral bands — SWIR22 (Shortwave Infrared 2), NIR (Near Infrared), Green, and SWIR16 (Shortwave Infrared 1) — and computed key spectral indices such as NDMI (Normalized Difference Moisture Index) and MNDWI (Modified Normalized Difference Water Index). These features capture surface moisture, vegetation, and water content characteristics that influence water quality variability. </p> <p align="justify"> In addition to Landsat features, we also incorporated the <b>Potential Evapotranspiration (PET)</b> variable from the <b>TerraClimate</b> dataset, which provides high-resolution global climate data. The PET feature captures the atmospheric demand for moisture, representing climatic conditions such as temperature, humidity, and radiation that influence surface water evaporation and thus affect water quality parameters. </p> <ul> <li>SWIR22 – Sensitive to surface moisture and turbidity variations in water bodies.</li> <li>NIR – Helps in identifying vegetation and suspended matter in water.</li> <li>Green – Useful for detecting water color and surface reflectance changes.</li> <li>SWIR16 – Provides information on surface dryness and sediment concentration.</li> <li>NDMI – Derived from NIR and SWIR16, indicates moisture and vegetation-water interaction.</li> <li>MNDWI – Derived from Green and SWIR22, effective for distinguishing open water areas and reducing built-up noise.</li> <li>PET – Extracted from the TerraClimate dataset, represents the potential evapotranspiration that influences hydrological and water quality dynamics.</li> </ul>

<h4 style="color:rgb(255, 0, 0)"><strong>Tip 1</strong></h4>  
<p align="justify">  
Participants are encouraged to experiment with different combinations of <b>Landsat</b> bands or even include data from other public satellite data sources. By creating mathematical combinations of bands, you can derive various spectral indices that capture surface and environmental characteristics.
</p>


<h3>Loading Pre-Extracted Landsat Data</h3>
<p align="justify">
In this notebook, we <b>load previously extracted Landsat data</b> from CSV files generated in a separate extraction notebook. This approach ensures a smoother and faster workflow, allowing participants to focus on data analysis and model development without waiting for time-consuming data retrieval.
</p>
<p align="justify">
Participants are expected to generate their own data extraction CSV files by running the dedicated Landsat extraction notebook. These CSV files can then be used here to smoothly run this benchmark notebook. Participants can refer to the extraction notebook to understand the API-based process, including how individual bands and indices like <b>NDMI</b> were computed. Using these pre-extracted CSV files simplifies preprocessing and is ideal for large-scale environmental and water quality analysis.
</p>


<h4 style="color:rgb(255, 0, 0)"><strong>Tip 2</strong></h4>
In the data extraction process (performed in the dedicated extraction notebooks), a 100 m focal buffer was applied around each sampling location rather than using a single point. Participants may explore creating different focal buffers around the locations (e.g., 50 m, 150 m, etc.) during extraction. For example, if a 50 m buffer was used for “Band 2”, the extracted CSV values would reflect the average of "Band 2" within 50 meters of each location. This approach can help reduce errors associated with spatial autocorrelation.


In [ ]:
landsat_train_features = pd.read_csv('/content/drive/MyDrive/landsat_features_training.csv')
landsat_train_features.head()

,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI
0,-28.760833,17.730278,02-01-2011,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595
1,-26.861111,28.884722,03-01-2011,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134
2,-26.450000,28.085833,03-01-2011,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805
3,-27.671111,27.236944,03-01-2011,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416
4,-27.356667,27.286389,03-01-2011,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683


<h3>Loading Pre-Extracted TerraClimate Data</h3>
<p align="justify">
In this notebook, we <b>load previously extracted TerraClimate data</b> from CSV files generated in a dedicated extraction notebook. This approach ensures a smoother and faster workflow, allowing participants to focus on data analysis and model development without waiting for time-consuming data retrieval.
</p>
<p align="justify">
Participants are expected to generate their own data extraction CSV files by running the dedicated TerraClimate extraction notebook. These CSV files can then be used here to smoothly run this benchmark notebook. Participants can refer to the extraction notebook to understand the API-based process, including how climate variables such as <b>Potential Evapotranspiration (PET)</b> were extracted. Using these pre-extracted CSV files ensures consistent, automated retrieval of high-resolution climate data that can be easily integrated with satellite-derived features for comprehensive environmental and hydrological analysis.
</p>


In [ ]:
Terraclimate_df = pd.read_csv('/content/drive/MyDrive/terraclimate_features_training.csv')
Terraclimate_df.head()

,Latitude,Longitude,Sample Date,pet
0,-28.760833,17.730278,02-01-2011,174.2
1,-26.861111,28.884722,03-01-2011,124.1
2,-26.450000,28.085833,03-01-2011,127.5
3,-27.671111,27.236944,03-01-2011,129.7
4,-27.356667,27.286389,03-01-2011,129.2


## Joining the predictor variables and response variables
Now that we have extracted our predictor variables, we need to join them onto the response variable . We use the function <i><b>combine_two_datasets</b></i> to combine the predictor variables and response variables.The <i><b>concat</b></i> function from pandas comes in handy here.

In [ ]:
import pandas as pd
import numpy as np

def combine_three_datasets(water_quality_df, landsat_df, terraclimate_df):
    '''
    Merges water quality data with satellite and climate data based on location and date.

    Arguments:
    water_quality_df -- The 'Left' table (Ground Truth)
    landsat_df       -- The 'Right' table (Satellite Features)
    terraclimate_df  -- The 'Right' table (Climate Features)
    '''

    # 1. STANDARDIZE DATES
    # Crucial: '01-02-2011' (str) and timestamp(2011-02-01) will NOT match.
    # We force all date columns to the same datetime format before merging.
    for df in [water_quality_df, landsat_df, terraclimate_df]:
        if 'Sample Date' in df.columns:
            df['Sample Date'] = pd.to_datetime(df['Sample Date'], format='%d-%m-%Y')

    # 2. DEFINE MERGE KEYS
    # These are the columns that must match exactly for data to be joined
    merge_keys = ['Latitude', 'Longitude', 'Sample Date']

    # 3. PERFORM LEFT JOINS
    # Step A: Attach Landsat to Water Quality
    merged_data = pd.merge(
        water_quality_df,
        landsat_df,
        on=merge_keys,
        how='left'
    )

    # Step B: Attach TerraClimate to the result
    merged_data = pd.merge(
        merged_data,
        terraclimate_df,
        on=merge_keys,
        how='left'
    )

    return merged_data

# --- STEP 2: LOAD DATASETS ---
# Make sure these file paths match exactly what you have in your folder
df_wq = pd.read_csv('/content/drive/MyDrive/water_quality_training_dataset.csv')
df_landsat = pd.read_csv('/content/drive/MyDrive/landsat_features_training.csv')
df_terra = pd.read_csv('/content/drive/MyDrive/terraclimate_features_training.csv')

# --- STEP 3: EXECUTE THE MERGE ---
# This creates the 'master_train' variable.
# If you skip this, the next step will give a NameError.
master_train = combine_three_datasets(df_wq, df_landsat, df_terra)

print(f"Data merged! Shape: {master_train.shape}")




Data merged! Shape: (9319, 13)


<h3>Handling Missing Values</h3>  
<p align="justify">  
Before model training, missing values in the dataset were carefully handled to ensure data consistency and prevent model bias. Numerical columns were imputed using their median values, maintaining the overall data distribution while minimizing the impact of outliers.  
</p>

In [ ]:
# --- STEP 4: FIX MISSING VALUES (IMPUTATION) ---
# This block uses 'master_train' which we just created above.

# Create a flag so the model knows we faked this data
master_train['is_cloudy'] = master_train['nir'].isnull().astype(int)

# Columns that need fixing
fill_cols = ['nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']

# Strategy A: Fill with the MEDIAN of that specific location (Lat/Lon)
# If a specific river spot usually has value X, we assume it's close to X today.
master_train[fill_cols] = master_train.groupby(['Latitude', 'Longitude'])[fill_cols].transform(lambda x: x.fillna(x.median()))

# Strategy B: Safety Net
# If a location has ONLY cloudy days (so the median is also NaN), fill with global median
master_train[fill_cols] = master_train[fill_cols].fillna(master_train[fill_cols].median())

# --- STEP 5: VERIFY ---
print("Missing values after fix:")
print(master_train.isnull().sum())



Missing values after fix:
Latitude                         0
Longitude                        0
Sample Date                      0
Total Alkalinity                 0
Electrical Conductance           0
Dissolved Reactive Phosphorus    0
nir                              0
green                            0
swir16                           0
swir22                           0
NDMI                             0
MNDWI                            0
pet                              0
is_cloudy                        0
dtype: int64


## Model Building

<p align="justify"> Now let us select the columns required for our model building exercise. We will consider only Band swir22, NDMI and MNDWI from the Landsat data and pet from Terraclimate dataset as our predictor variables. It does not make sense to use latitude and longitude as predictor variables, as they do not have any direct impact on predicting the water quality parameters.</p>


In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import numpy as np

# --- STEP 1: PREPARE DATA ---
# Define your targets
target_cols = ['Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']

# Define your features (Drop targets and non-numeric columns like Date)
# We also drop 'Latitude' and 'Longitude' if we don't want the model to memorize coordinates
# But keeping them is often okay for regional trends. Let's keep them for now.
drop_cols = target_cols + ['Sample Date']
X = master_train.drop(columns=drop_cols)
y = master_train[target_cols]

# Create the "Group" column (Unique ID for each location)
groups = master_train['Latitude'].astype(str) + "_" + master_train['Longitude'].astype(str)

# --- STEP 2: SETUP GROUP K-FOLD ---
# 5 Splits means we train on 80% of locations and test on 20%, 5 times.
gkf = GroupKFold(n_splits=5)

# Dictionaries to store scores for each target
cv_scores = {target: [] for target in target_cols}
models = {} # To store the last fold's models (optional)

print("Starting Cross-Validation...")

# --- STEP 3: THE TRAINING LOOP ---
# Updated Training Loop with Log-Transformation
for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    for target in target_cols:
        model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

        # --- CHANGE 1: TRAIN ON LOG ---
        # We use np.log1p (log of 1+x) to handle zeros safely
        y_train_log = np.log1p(y_train[target])
        model.fit(X_train, y_train_log)

        # --- CHANGE 2: PREDICT & CONVERT BACK ---
        # Predict the log value
        preds_log = model.predict(X_val)
        # Convert back to normal scale using np.expm1 (exponential minus 1)
        preds = np.expm1(preds_log)

        # Evaluate on the REAL values (not the log values)
        score = r2_score(y_val[target], preds)
        cv_scores[target].append(score)

    print(f"Fold {fold+1} completed.")

# --- STEP 4: REPORT RESULTS ---
print("\n--- Final Cross-Validation Results ---")
overall_score = 0
for target in target_cols:
    mean_score = np.mean(cv_scores[target])
    print(f"{target}: {mean_score:.4f}")
    overall_score += mean_score

print(f"\nAVERAGE COMPETITION SCORE: {overall_score / 3:.4f}")

Starting Cross-Validation...
Fold 1 completed.
Fold 2 completed.
Fold 3 completed.
Fold 4 completed.
Fold 5 completed.

--- Final Cross-Validation Results ---
Total Alkalinity: 0.1721
Electrical Conductance: 0.2614
Dissolved Reactive Phosphorus: -0.0363

AVERAGE COMPETITION SCORE: 0.1324


<h4 style="color:rgb(255, 0, 0)"><strong>Tip 3</strong></h4>
<p align="justify">We are developing individual models for each water quality parameter using a common set of features: SWIR22, NDMI, MNDWI, and PET. However, participants are encouraged to experiment with different feature combinations to build more robust machine learning models.</p>

## Helper Functions
### Train and Test Split
<p align="justify">We will now split the data into 70% training data and 30% test data. Scikit-learn alias “sklearn” is a robust library for machine learning in Python. The scikit-learn library has a <i><b>model_selection</b></i> module in which there is a splitting function <i><b>train_test_split</b></i>. You can use the same.</p>

### Feature Scaling

<p align="justify"> Before initiating the model training we may have to execute different data pre-processing steps. Here we are demonstrating the scaling of 'swir22','NDMI','MNDWI','pet' variable by using Standard Scaler.</p>

<p align = "justify">Feature Scaling is a data preprocessing step for numerical features. Many machine learning algorithms like Gradient descent methods, KNN algorithm, linear and logistic regression, etc. require data scaling to produce good results. Scikit learn provides functions that can be used to apply data scaling. Here we are using Standard Scaler. The idea behind Standard Scaler is that it will transform your data such that its distribution will have a mean value 0 and standard deviation of 1.</p>

### Model Training
<p align="justify">
Now that we have the data in a format suitable for machine learning, we can begin training our models. In this demonstration notebook, we will build three separate regression models — one for each target water quality parameter: Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus.
Each model will be trained independently to capture the unique relationships between the satellite-derived features and each parameter.
</p>

<p align="justify">
We will use the Random Forest Regressor from the scikit-learn library to build our models. Scikit-learn provides a wide range of regression algorithms with extensive parameter tuning and customization capabilities.
</p>

<p align="justify">
For model training, the predictor variables (e.g., SWIR22, NDMI, MNDWI, and pet) will be stored in an array X, and the response variable (one of the three water quality parameters) will be stored in an array Y.
It is important not to include the response variable in X. Additionally, since latitude, longitude, and sample date are only used for spatial and temporal reference, they will be excluded from the predictor variables during model training.
</p>

### Model Evaluation
<p align="justify">
Now that we have trained our models for the three water quality parameters, the next step is to evaluate their performance. Each regression model for Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus is assessed using the R² score and the Root Mean Square Error (RMSE). The R² score measures how well the model explains the variance in the observed values, while RMSE quantifies the average magnitude of prediction errors. Together, these metrics help determine how effectively each model captures variations in water quality across different locations and sampling dates. Scikit-learn provides built-in functions to compute these metrics, and participants may explore additional evaluation methods or custom metrics as needed.</p>


<h4 style="color:rgb(255, 0, 0)"><strong>Tip 4</strong></h4>
<p align="justify">There are many data preprocessing methods available, which might help to improve the model performance. Participants should explore various suitable preprocessing methods as well as different machine learning algorithms to build a robust model.</p>

In [ ]:
def split_data(X, y, test_size=0.3, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

def scale_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, scaler

def train_model(X_train_scaled, y_train):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)
    return model

def evaluate_model(model, X_scaled, y_true, dataset_name="Test"):
    y_pred = model.predict(X_scaled)
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"\n{dataset_name} Evaluation:")
    print(f"R²: {r2:.3f}")
    print(f"RMSE: {rmse:.3f}")
    return y_pred, r2, rmse

<div class="section">
  <h2>Model Workflow (Pipeline)</h2>
  <p align="justify">
    The complete model development process follows a structured pipeline to ensure consistency, reproducibility, and clarity.
    Each stage in the workflow is modularized into independent functions, which can be reused for different water quality parameters.
    This modular approach helps streamline the process and makes the workflow easily adaptable to new datasets or parameters in the future.
  </p>

  <p align="justify">
    The pipeline automates the sequence of steps — from data preparation to evaluation — for each target parameter.
    The same set of predictor variables is used, while the response variable changes for each of the three targets:
    <i>Total Alkalinity (TA)</i>, <i>Electrical Conductance (EC)</i>, and <i>Dissolved Reactive Phosphorus (DRP)</i>.
    By maintaining a consistent framework, comparisons across models remain fair and interpretable.
  </p>


In [ ]:
def run_pipeline(X, y, param_name="Parameter"):
    print(f"\n{'='*60}")
    print(f"Training Model for {param_name}")
    print(f"{'='*60}")

    # Split data
    X_train, X_test, y_train, y_test = split_data(X, y)

    # Scale
    X_train_scaled, X_test_scaled, scaler = scale_data(X_train, X_test)

    # Train
    model = train_model(X_train_scaled, y_train)

    # Evaluate (in-sample)
    y_train_pred, r2_train, rmse_train = evaluate_model(model, X_train_scaled, y_train, "Train")

    # Evaluate (out-sample)
    y_test_pred, r2_test, rmse_test = evaluate_model(model, X_test_scaled, y_test, "Test")

    # Return summary
    results = {
        "Parameter": param_name,
        "R2_Train": r2_train,
        "RMSE_Train": rmse_train,
        "R2_Test": r2_test,
        "RMSE_Test": rmse_test
    }
    return model, scaler, pd.DataFrame([results])

### Model Training and Evaluation for Each Parameter

<p align="justify">In this step, we apply the complete modeling pipeline to each of the three selected water quality parameters — Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus. The input feature set (<code>X</code>) remains the same across all three models, while the target variable (<code>y</code>) changes for each parameter. For every parameter, the <code>run_pipeline()</code> function is executed, which handles data preprocessing, model training, and both in-sample and out-of-sample evaluation. This ensures a consistent workflow and allows for a fair comparison of model performance across different water quality indicators.</p>

In [ ]:
X = wq_data.drop(columns=['Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus'])

y_TA = wq_data['Total Alkalinity']
y_EC = wq_data['Electrical Conductance']
y_DRP = wq_data['Dissolved Reactive Phosphorus']

model_TA, scaler_TA, results_TA = run_pipeline(X, y_TA, "Total Alkalinity")
model_EC, scaler_EC, results_EC = run_pipeline(X, y_EC, "Electrical Conductance")
model_DRP, scaler_DRP, results_DRP = run_pipeline(X, y_DRP, "Dissolved Reactive Phosphorus")



Training Model for Total Alkalinity

Train Evaluation:
R²: 0.903
RMSE: 23.124

Test Evaluation:
R²: 0.546
RMSE: 50.877

Training Model for Electrical Conductance

Train Evaluation:
R²: 0.918
RMSE: 98.029

Test Evaluation:
R²: 0.585
RMSE: 220.214

Training Model for Dissolved Reactive Phosphorus

Train Evaluation:
R²: 0.882
RMSE: 17.455

Test Evaluation:
R²: 0.529
RMSE: 35.182


### Model Performance Summary

<p align="justify">After training and evaluating the models for each water quality parameter, the individual performance metrics are combined into a single summary table. This table consolidates the R² and RMSE values for both in-sample and out-of-sample evaluations, enabling an easy comparison of model performance across Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus. Such a summary provides a quick overview of how well each model captures the variability in the respective parameter and highlights any differences in predictive accuracy.</p>

In [ ]:
results_summary = pd.concat([results_TA, results_EC, results_DRP], ignore_index=True)
results_summary


,Parameter,R2_Train,RMSE_Train,R2_Test,RMSE_Test
0,Total Alkalinity,0.903272,23.123742,0.545544,50.876902
1,Electrical Conductance,0.917848,98.028822,0.584648,220.213714
2,Dissolved Reactive Phosphorus,0.882169,17.454757,0.529145,35.181776


## Submission

<p align="justify">Once you are satisfied with your model’s performance, you can proceed to make predictions for unseen data. To do this, use your trained model to estimate the concentrations of the target water quality parameters — Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus — for a set of test locations provided in the "Submission_template.csv" file. The predicted results can then be uploaded to the challenge platform for evaluation.</p>

In [ ]:
#Reading the coordinates for the submission
test_file = pd.read_csv('/EY dataset/submission_template.csv')
test_file.head()

,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-32.043333,27.822778,01-09-2014,NaN,NaN,NaN
1,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN
2,-32.991639,27.640028,07-05-2015,NaN,NaN,NaN
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN
4,-32.000556,28.581667,01-10-2014,NaN,NaN,NaN


<p align="justify">
Similarly, participants can use the <b>Landsat</b> and <b>TerraClimate</b> data extraction demonstration notebooks to produce feature CSVs for their <b>validation</b> data. For convenience, we have already computed and saved example validation outputs as <code>landsat_features_val_V3.csv</code> and <code>Terraclimate_val_df_v3.csv</code>. Participants should save their own extracted files in the same format and column schema; doing so will allow this benchmark notebook to load the validation features directly and run smoothly.
</p>


In [ ]:
landsat_val_features = pd.read_csv('/EY dataset/landsat_features_validation.csv')
landsat_val_features.head()

,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI
0,-32.043333,27.822778,01-09-2014,15229.0,12868.0,14797.0,12421.0,0.014388,-0.069727
1,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN,NaN,NaN,NaN
2,-32.991639,27.640028,07-05-2015,16221.0,9304.5,12536.5,9958.0,0.128123,-0.147979
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN,NaN,NaN,NaN
4,-32.000556,28.581667,01-10-2014,9125.0,11100.5,9455.0,8711.0,-0.017761,0.080052


In [ ]:
Terraclimate_val_df = pd.read_csv('/EY dataset/terraclimate_features_validation.csv')
Terraclimate_val_df.head()

,Latitude,Longitude,Sample Date,pet
0,-32.043333,27.822778,01-09-2014,161.90001
1,-33.329167,26.077500,16-09-2015,177.60000
2,-32.991639,27.640028,07-05-2015,158.40001
3,-34.096389,24.439167,07-02-2012,130.00000
4,-32.000556,28.581667,01-10-2014,152.50000


In [ ]:
#Consolidate all the extracted bands and features in a single dataframe
val_data = pd.DataFrame({
    'Longitude': landsat_val_features['Longitude'].values,
    'Latitude': landsat_val_features['Latitude'].values,
    'Sample Date': landsat_val_features['Sample Date'].values,
    'nir': landsat_val_features['nir'].values,
    'green': landsat_val_features['green'].values,
    'swir16': landsat_val_features['swir16'].values,
    'swir22': landsat_val_features['swir22'].values,
    'NDMI': landsat_val_features['NDMI'].values,
    'MNDWI': landsat_val_features['MNDWI'].values,
    'pet': Terraclimate_val_df['pet'].values,
})

In [ ]:
# Impute the missing values
val_data = val_data.fillna(val_data.median(numeric_only=True))

In [ ]:
# Extracting specific columns (B01, B06, and NDVI) from the validation dataset
submission_val_data=val_data.loc[:,['swir22','NDMI','MNDWI','pet']]
submission_val_data.head()

,swir22,NDMI,MNDWI,pet
0,12421.0,0.014388,-0.069727,161.90001
1,9973.0,0.081427,-0.130571,177.60000
2,9958.0,0.128123,-0.147979,158.40001
3,9973.0,0.081427,-0.130571,130.00000
4,8711.0,-0.017761,0.080052,152.50000


In [ ]:
submission_val_data.shape

(200, 4)

In [ ]:
# --- Predicting for Total Alkalinity ---
X_sub_scaled_TA = scaler_TA.transform(submission_val_data)
pred_TA_submission = model_TA.predict(X_sub_scaled_TA)

# --- Predicting for Electrical Conductance ---
X_sub_scaled_EC = scaler_EC.transform(submission_val_data)
pred_EC_submission = model_EC.predict(X_sub_scaled_EC)

# --- Predicting for Dissolved Reactive Phosphorus ---
X_sub_scaled_DRP = scaler_DRP.transform(submission_val_data)
pred_DRP_submission = model_DRP.predict(X_sub_scaled_DRP)

In [ ]:
submission_df = pd.DataFrame({
    'Longitude': test_file['Longitude'].values,
    'Latitude': test_file['Latitude'].values,
    'Sample Date': test_file['Sample Date'].values,
    'Total Alkalinity': pred_TA_submission,
    'Electrical Conductance': pred_EC_submission,
    'Dissolved Reactive Phosphorus': pred_DRP_submission
})

In [ ]:
#Displaying the sample submission dataframe
submission_df.head()

,Longitude,Latitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,27.822778,-32.043333,01-09-2014,114.558899,315.525271,25.235333
1,26.077500,-33.329167,16-09-2015,155.383281,647.889489,60.455833
2,27.640028,-32.991639,07-05-2015,61.108269,723.033983,29.445333
3,24.439167,-34.096389,07-02-2012,71.718494,234.056986,13.726667
4,28.581667,-32.000556,01-10-2014,108.912650,312.262200,30.210000


In [ ]:
#Dumping the predictions into a csv file.
submission_df.to_csv("submission.csv",index = False)

FileNotFoundError: [Errno 2] No such file or directory: 'submission.csv'

### Upload submission file on platform

Upload the submission.csv on the <a href ="https://challenge.ey.com">platform</a> to get score generated on scoreboard.

## Conclusion

<div align ="justify">Now that you have learned a basic approach to model training, it’s time to try your own approach! Feel free to modify any of the functions presented in this notebook. We look forward to seeing your version of the model and the results. Best of luck with the challenge!</div>